# ETL Analysis

In [45]:
from supabase import create_client, Client
import os
import pandas as pd
from dotenv import load_dotenv
import numpy as np
load_dotenv()
SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_KEY = os.getenv("SUPABASE_KEY")
supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)
data = supabase.table("telco_customer_churn_data").select('*').execute()
df = pd.DataFrame(data.data)

In [46]:
df.head()

,id,tenure,monthlycharges,totalcharges,churn,internetservice,contract,paymentmethod,tenure_group,monthly_charge_segment,has_internet_service,is_multi_line_user,contract_type_code
0,1,1,29.85,29.85,No,DSL,Month-to-month,Electronic check,New,1,0,0,None
1,2,34,56.95,1889.50,No,DSL,One year,Mailed check,Regular,1,0,1,None
2,3,2,53.85,108.15,Yes,DSL,Month-to-month,Mailed check,New,1,0,0,None
3,4,45,42.30,1840.75,No,DSL,One year,Bank transfer (automatic),Loyal,1,0,1,None
4,5,2,70.70,151.65,Yes,Fiber optic,Month-to-month,Electronic check,New,1,0,0,None


In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      1000 non-null   int64  
 1   tenure                  1000 non-null   int64  
 2   monthlycharges          1000 non-null   float64
 3   totalcharges            1000 non-null   float64
 4   churn                   1000 non-null   object 
 5   internetservice         1000 non-null   object 
 6   contract                1000 non-null   object 
 7   paymentmethod           1000 non-null   object 
 8   tenure_group            997 non-null    object 
 9   monthly_charge_segment  1000 non-null   int64  
 10  has_internet_service    1000 non-null   int64  
 11  is_multi_line_user      1000 non-null   int64  
 12  contract_type_code      0 non-null      object 
dtypes: float64(2), int64(5), object(6)
memory usage: 101.7+ KB


In [48]:
df.isnull().sum()

id                           0
tenure                       0
monthlycharges               0
totalcharges                 0
churn                        0
internetservice              0
contract                     0
paymentmethod                0
tenure_group                 3
monthly_charge_segment       0
has_internet_service         0
is_multi_line_user           0
contract_type_code        1000
dtype: int64

In [49]:
df['tenure_group'] = df['tenure_group'].fillna('Unknown')

In [50]:
# Calcuating the metrics
# Churn percentage
churn_percentage = (df['churn'].value_counts(normalize=True) * 100).to_dict()
print("Churn Percentage:\n", churn_percentage)

Churn Percentage:
 {'No': 74.4, 'Yes': 25.6}


In [51]:
# average monthly charges per contract
avg_monthly_charges = df.groupby('contract')['monthly_charge_segment'].mean().to_dict()
print("\nAverage Monthly Charges per Contract:\n", avg_monthly_charges)



Average Monthly Charges per Contract:
 {'Month-to-month': 0.8670309653916212, 'One year': 0.7696078431372549, 'Two year': 0.6639676113360324}


In [52]:
#     Count of new, regular, loyal, champion customers
customer_segments = df['tenure_group'].value_counts().to_dict()
print("\nCustomer Segments Count:\n", customer_segments)


Customer Segments Count:
 {'New': 318, 'Regular': 268, 'Champion': 209, 'Loyal': 202, 'Unknown': 3}


In [53]:
#     Internet service distribution
internet_service_distribution = df['internetservice'].value_counts(normalize=True).to_dict()
print("\nInternet Service Distribution:\n", internet_service_distribution)


Internet Service Distribution:
 {'Fiber optic': 0.468, 'DSL': 0.329, 'No': 0.203}


In [54]:
#     Pivot table: Churn vs Tenure Group
churn_tenure_pivot = pd.pivot_table(df, values='id', index='tenure_group', columns='churn', aggfunc='count', fill_value=0)
print("\nChurn vs Tenure Group Pivot Table:\n", churn_tenure_pivot)


Churn vs Tenure Group Pivot Table:
 churn          No  Yes
tenure_group          
Champion      196   13
Loyal         169   33
New           164  154
Regular       212   56
Unknown         3    0
